In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

import warnings
warnings.filterwarnings("ignore")
sns.set(style="whitegrid")


## Veri Seti
Bu örnekte doğrudan dosyaya gömülü haliyle kullanılmaktadır.

In [ ]:
# Örnek veri seti: housing.csv (örnek amaçlı veri yükleniyor)
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing()
df = pd.DataFrame(data.data, columns=data.feature_names)
df["MedHouseVal"] = data.target
df.head()


## Veri Keşfi ve Görselleştirme

In [ ]:
df.describe()


In [ ]:

plt.figure(figsize=(8, 5))
sns.histplot(df["MedHouseVal"], bins=30, kde=True)
plt.title("Hedef Değişken (MedHouseVal) Dağılımı")
plt.show()


## Veri Setini Eğitme ve Test Olarak Ayırma

In [ ]:
X = df.drop("MedHouseVal", axis=1)
y = df["MedHouseVal"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Model Eğitimi ve Performans Karşılaştırması

In [ ]:
def evaluate_model(model, name):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    r2 = r2_score(y_test, predictions)
    
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=y_test, y=predictions, alpha=0.4)
    plt.xlabel("Gerçek Değer")
    plt.ylabel("Tahmin")
    plt.title(f"{name} Tahmin vs Gerçek")
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.show()
    
    return {"model": name, "rmse": rmse, "r2": r2}


In [ ]:
results = []

lr = LinearRegression()
results.append(evaluate_model(lr, "Linear Regression"))

dt = DecisionTreeRegressor(max_depth=5)
results.append(evaluate_model(dt, "Decision Tree"))

rf = RandomForestRegressor(n_estimators=100, random_state=42)
results.append(evaluate_model(rf, "Random Forest"))


## Decision Tree Görselleştirme

In [ ]:
plt.figure(figsize=(20, 10))
plot_tree(dt, feature_names=X.columns, filled=True, rounded=True, fontsize=8)
plt.title("Decision Tree Yapısı")
plt.show()


## Model Karşılaştırması

In [ ]:
results_df = pd.DataFrame(results)
plt.figure(figsize=(10, 5))
sns.barplot(data=results_df, x="model", y="r2")
plt.title("Modellerin R² Karşılaştırması")
plt.ylabel("R Kare Skoru")
plt.show()

plt.figure(figsize=(10, 5))
sns.barplot(data=results_df, x="model", y="rmse")
plt.title("Modellerin RMSE Karşılaştırması")
plt.ylabel("Root Mean Squared Error")
plt.show()


## Sonuç
Bu çalışmada kullanılan 3 model arasında genellikle Random Forest en iyi performansı göstermektedir.
- Linear Regression, doğrusal varsayımlar altında çalışır.
- Decision Tree, veriyi bölerek tahmin yapar ancak aşırı öğrenme riski vardır.
- Random Forest, çoklu ağaçlardan faydalanarak genelleme gücünü artırır.